In [1]:
import requests as rq
import pandas as pd


In [2]:
df_aerodromo = pd.read_excel('aeroportos_BR.xlsx')

In [3]:
df_top_10_aeroportos = pd.read_parquet('top_10_aeroportos.parquet')

In [4]:
lst_IATA = list(df_top_10_aeroportos["aeroporto_nome"].unique())

In [5]:
# Seleciona os 10 principais Aeroportos do país
df_aerodromo = df_aerodromo[
    df_aerodromo["IATA"].isin(lst_IATA)
]

In [6]:
aeroportos = zip(df_aerodromo["IATA"], df_aerodromo["ICAO"])

### [Geocoding API](https://open-meteo.com/en/docs/geocoding-api#api-documentation)

In [7]:
# Obtem as latitudades e longitudes dos Aeroportos

info_areo = []
aero_404 = []

for IATA, ICAO in aeroportos:
    
    url = "https://geocoding-api.open-meteo.com/v1/search?name={}&language=pt&count=10".format(ICAO)
    response = rq.get(url)
    data = response.json()
    
    # Validando se a resposta da API é válida
    if (response.status_code != 400) and (len(data) > 1):
        tmp = pd.DataFrame(data['results'])
        tmp["IATA"] = IATA
        tmp["ICAO"] = ICAO
        # Filter only airport and BR
        tmp = tmp[
            (tmp["country_code"] == "BR") & 
            (tmp["feature_code"] == "AIRP")
        ]
        info_areo.append(tmp)
    else:
        aero_404.append(IATA)

print("Não locatizado:\n", aero_404)
geo_aero = pd.concat(info_areo)
df_geo_aero = geo_aero[["ICAO","IATA", "latitude", "longitude", "elevation"]]

Não locatizado:
 []


In [8]:
df_geo_aero.head()

,ICAO,IATA,latitude,longitude,elevation
0,SBBR,BSB,-15.87110,-47.91862,1060.0
0,SBSP,CGH,-23.62754,-46.65597,801.0
0,SBSV,SSA,-12.91099,-38.33104,19.0
0,SBPA,POA,-29.99443,-51.17143,3.0
0,SBKP,VCP,-23.00738,-47.13452,661.0


### [Historical Weather API](https://open-meteo.com/en/docs/historical-weather-api)

In [9]:
start_date = "2022-01-01"
end_date = "2022-12-31"

endpoint = 'https://archive-api.open-meteo.com/v1/archive?'

params = '&hourly=temperature_2m,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,'\
          'windspeed_10m,windspeed_100m&daily=weathercode&timezone=America%2FSao_Paulo'

In [10]:
meteo_aero = []

for index, row in df_geo_aero.iterrows():
    filters = f"latitude={row['latitude']}&longitude={row['longitude']}&start_date={start_date}&end_date={end_date}"
    url = endpoint + filters + params
    print(url)
    response = rq.get(url)
    data = response.json()
    
    # Validando se a resposta da API é válida
    if (response.status_code != 400) and (len(data) > 1):
        tmp = pd.DataFrame(data['hourly'])
        
        tmp[["date", "hour"]] = tmp["time"].str.split('T', 1, expand=True)
        tmp['elevation'] = int(data['elevation'])
        tmp['weathercode'] = data['daily']['weathercode'][0]
        tmp["IATA"] = row['IATA']
        
        meteo_aero.append(tmp)
        
df_tempo = pd.concat(meteo_aero)

https://archive-api.open-meteo.com/v1/archive?latitude=-15.8711&longitude=-47.91862&start_date=2022-01-01&end_date=2022-12-31&hourly=temperature_2m,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m&daily=weathercode&timezone=America%2FSao_Paulo
https://archive-api.open-meteo.com/v1/archive?latitude=-23.62754&longitude=-46.65597&start_date=2022-01-01&end_date=2022-12-31&hourly=temperature_2m,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m&daily=weathercode&timezone=America%2FSao_Paulo
https://archive-api.open-meteo.com/v1/archive?latitude=-12.91099&longitude=-38.33104&start_date=2022-01-01&end_date=2022-12-31&hourly=temperature_2m,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m&daily=weathercode&timezone=America%2FSao_Paulo
https://archive-api.open-meteo.com/v1/archive?latitude=-29.99443&longitude=-51.17143&start_date=20

In [11]:
df_tempo.head()

,time,temperature_2m,precipitation,rain,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,windspeed_100m,date,hour,elevation,weathercode,IATA
0,2022-01-01T00:00,18.8,1.1,1.1,100,51,85,100,9.3,18.6,2022-01-01,00:00,1057,63,BSB
1,2022-01-01T01:00,18.4,0.9,0.9,100,95,88,99,9.7,18.3,2022-01-01,01:00,1057,63,BSB
2,2022-01-01T02:00,18.1,0.5,0.5,100,95,96,98,7.5,15.7,2022-01-01,02:00,1057,63,BSB
3,2022-01-01T03:00,18.1,0.2,0.2,100,86,71,99,5.9,13.6,2022-01-01,03:00,1057,63,BSB
4,2022-01-01T04:00,18.2,0.2,0.2,100,98,72,98,5.8,12.0,2022-01-01,04:00,1057,63,BSB


In [12]:
df_tempo.to_parquet('meteorologia.parquet')